In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import os
os.chdir('..')
from datetime import datetime, timedelta
import lightgbm as lgb
import joblib
import matplotlib.cm as cm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
import lightgbm as lgb
import joblib
import matplotlib.cm as cm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [3]:
# =============================
# MultiColumnLabelEncoder
# =============================
class MultiColumnLabelEncoder:
    def __init__(self, categorical_cols, unknown_token="unknown"):
        self.categorical_cols = categorical_cols
        self.unknown_token = unknown_token
        self.encoders = {}

    def fit(self, df: pd.DataFrame):
        for col in self.categorical_cols:
            le = LabelEncoder()
            values = df[col].fillna(self.unknown_token).astype(str)
            classes = values.unique().tolist()
            if self.unknown_token not in classes:
                classes.append(self.unknown_token)
            le.fit(classes)
            self.encoders[col] = le
        return self

    def transform(self, df: pd.DataFrame):
        df = df.copy()
        for col, le in self.encoders.items():
            values = df[col].fillna(self.unknown_token).astype(str)
            # Những giá trị không nằm trong classes_ được map sang unknown_token
            values = values.where(values.isin(le.classes_), self.unknown_token)
            df[col] = le.transform(values)
        return df

    def fit_transform(self, df: pd.DataFrame):
        self.fit(df)
        return self.transform(df)

    def save(self, path: str):
        joblib.dump({
            "categorical_cols": self.categorical_cols,
            "unknown_token": self.unknown_token,
            "encoders": self.encoders
        }, path)

    @classmethod
    def load(cls, path: str):
        data = joblib.load(path)
        obj = cls(
            categorical_cols=data["categorical_cols"],
            unknown_token=data["unknown_token"]
        )
        obj.encoders = data["encoders"]
        return obj

# MERGE DATA TRAINING

In [4]:
model_dir = "models_1578_csv"
os.makedirs(model_dir, exist_ok=True)

In [5]:
def load_data(path):
    data_df = pd.read_csv(path)
    return data_df

In [6]:
dataset_all_building = "data_1578_csv"
list_building = [
    f for f in os.listdir(dataset_all_building)
]
print(list_building)

['Fox_lodging_Morris', 'Eagle_office_Sonya', 'Fox_utility_Marian', 'Fox_office_Carson', 'Eagle_office_Elvis', 'Eagle_public_Henry', 'Eagle_assembly_Candice', 'Eagle_office_Ryan', 'Eagle_assembly_Josie', 'Fox_assembly_Cathy', 'Eagle_education_Shanna', 'Eagle_office_Efrain', 'Fox_education_Melinda', 'Fox_education_Theodore', 'Eagle_office_Isidro', 'Fox_retail_Manie', 'Fox_education_Leota', 'Fox_education_Nilda', 'Eagle_office_Donovan', 'Fox_education_Lilly', 'Eagle_lodging_Stephanie', 'Eagle_office_Yadira', 'Fox_education_Suzan', 'Fox_education_Stacia', 'Fox_education_Geoffrey', 'Eagle_office_Jeff', 'Fox_education_Andre', 'Fox_public_Martin', 'Eagle_public_Minnie', 'Eagle_education_Luther', 'Fox_education_Delma', 'Fox_education_Kim', 'Fox_lodging_Stephen', 'Fox_lodging_Jina', 'Fox_education_Wendell', 'Fox_office_Bernard', 'Fox_lodging_Warren', 'Fox_office_Rowena', 'Eagle_office_Francis', 'Fox_education_Marlana', 'Fox_health_Lorena', 'Fox_education_Jaclyn', 'Fox_education_Sterling', 'Fox_

In [7]:

# =============================
# CONFIG
# =============================
CAT_COLS = ["primaryspaceusage", "site_id", "building_id"]  # sửa typo

# =============================
# LOAD & CONCAT TRAIN DATA
# =============================
df_train_all = pd.DataFrame()
for building in list_building:
    path = f"{dataset_all_building}/{building}/train.csv"
    if os.path.exists(path):
        df = load_data(path)  # bạn định nghĩa hàm load_data()
        print("*"*10)
        print(f"Process building: {building}- {len(df)}", )
        df_train_all = pd.concat([df_train_all, df], ignore_index=True)
        print("Total: ", len(df_train_all))
    else:
        print(f"Train file not found for {building}")

df_train_all.to_csv(f"{dataset_all_building}/train.csv", index=False)

# =============================
# FIT ENCODER TRÊN TRAIN
# =============================
encoder = MultiColumnLabelEncoder(CAT_COLS)
df_train_encoded = encoder.fit_transform(df_train_all)
df_train_encoded.to_csv(f"{dataset_all_building}/train_encode.csv", index=False)
encoder.save(f"{model_dir}/categorical_encoder.pkl")  # save encoder sau khi fit

# =============================
# LOAD & CONCAT TEST DATA
# =============================
df_test_all = pd.DataFrame()
for building in list_building:
    path = f"{dataset_all_building}/{building}/test.csv"
    if os.path.exists(path):
        df = load_data(path)
        df_test_all = pd.concat([df_test_all, df], ignore_index=True)
        print("Total test: ", len(df_test_all))
    else:
        print(f"Test file not found for {building}")

df_test_all.to_csv(f"{dataset_all_building}/test.csv", index=False)

# =============================
# TRANSFORM TEST DATA
# =============================
encoder = MultiColumnLabelEncoder.load(f"{model_dir}/categorical_encoder.pkl")  # load encoder train
df_test_encoded = encoder.transform(df_test_all)  # chỉ transform
df_test_encoded.to_csv(f"{dataset_all_building}/test_encode.csv", index=False)

print("Train/Test encoding done ✅")

**********
Process building: Fox_lodging_Morris- 14542
Total:  14542
**********
Process building: Eagle_office_Sonya- 14081
Total:  28623
**********
Process building: Fox_utility_Marian- 14224
Total:  42847
**********
Process building: Fox_office_Carson- 14565
Total:  57412
**********
Process building: Eagle_office_Elvis- 13869
Total:  71281
**********
Process building: Eagle_public_Henry- 14414
Total:  85695
**********
Process building: Eagle_assembly_Candice- 13794
Total:  99489
**********
Process building: Eagle_office_Ryan- 14105
Total:  113594
**********
Process building: Eagle_assembly_Josie- 13926
Total:  127520
**********
Process building: Fox_assembly_Cathy- 14202
Total:  141722
**********
Process building: Eagle_education_Shanna- 14072
Total:  155794
**********
Process building: Eagle_office_Efrain- 13814
Total:  169608
**********
Process building: Fox_education_Melinda- 14564
Total:  184172
**********
Process building: Fox_education_Theodore- 14057
Total:  198229
**********
